In [1]:
import numpy as np
import nrrd
from os.path import *
from pylab import *
from JSONread import *
from density_function import *
import json

DATA_FOLDER = "data/"
OUTPUT_FOLDER = "output/"

In [2]:
annotation, h = nrrd.read(join(DATA_FOLDER, "annotations.nrrd"))
neu_dens, h = nrrd.read(join(DATA_FOLDER, "neu_density.nrrd"))
num_neurons = json.loads(open(join(DATA_FOLDER, "neuron_counts.json"), "r").read())
jsontextfile = open(join(DATA_FOLDER, "brain_regions.json"), "r")
jsoncontent = json.loads(jsontextfile.read())
search_children(jsoncontent['msg'][0])
rv = json.loads(open(join(DATA_FOLDER, "volumes_25.json"), "r").read())
voxel_volume = 25**3/1.0e9

# Computing densities

In [3]:
uniques = find_unique_regions(annotation, 
                        id_to_region_dictionary_ALLNAME, 
                        region_dictionary_to_id_ALLNAME,
                        region_dictionary_to_id_ALLNAME_parent, 
                        name2allname)

children, order_ = find_children(uniques, id_to_region_dictionary_ALLNAME, is_leaf,
                                  region_dictionary_to_id_ALLNAME_parent, 
                                  region_dictionary_to_id_ALLNAME)
uniques = uniques[np.argsort(order_)] # order from leaf to biggest regions

In [4]:
####### Choose Markers and their files
marker_names = ["GAD"] # Careful with the order: should work with the literature file
volume_files = ["479_gad1_expr.npy"]
literature_file = join(DATA_FOLDER, "gaba_papers.xlsx")
sheet_indices = [0, 1]
num_first_row = 1
column_name = 0
columns_mean = [[1]]
num_marker = len(marker_names)

In [5]:
# Loading results of the fitting pipeline
alphas = np.zeros((num_marker, 3))
std_fitt = np.zeros((num_marker, 3))
for i_marker, name in enumerate(marker_names):
    jsoncontent = json.loads(open(join(OUTPUT_FOLDER, "fitting_" + name + ".json"), "r").read())
    for i_key, key in enumerate(["Cerebellum", "Isocortex", "Rest"]):
        if key in jsoncontent["alphas"].keys():
            alphas[i_marker][i_key] = jsoncontent["alphas"][key]
            std_fitt[i_marker][i_key] = jsoncontent["std"][key]

In [6]:
markers_intensity = np.zeros((num_marker,) + annotation.shape)
for i_marker, filename in enumerate(volume_files):
    markers_intensity[i_marker] = np.load(join(DATA_FOLDER, filename))

In [7]:
names, mean_literature, down_std_literature, up_std_literature, convert = read_densities_sheet(
                    literature_file, region_keys, columns_mean,
                    sheet_indices, num_first_row, column_name)

In [8]:
# In case of multiple literature values for a region, 
# taking the mean of the means and min and max for stds
unames, ucounts = np.unique(names, return_counts=True)
mean_literature = mean_literature[0]
down_std_literature = down_std_literature[0]
up_std_literature = up_std_literature[0]
for name in unames[np.where(ucounts>1)]:
    filter_ = names==name
    names = np.delete(names, np.where(filter_))
    names = np.append(names, name)
    
    GAD = np.mean(mean_literature[filter_])
    mean_literature = np.delete(mean_literature, np.where(filter_))
    mean_literature = np.append(mean_literature, GAD)
    
    down_std = np.min(down_std_literature[filter_])
    down_std_literature = np.delete(down_std_literature, np.where(filter_))
    down_std_literature = np.append(down_std_literature, down_std)
    
    up_std = np.max(up_std_literature[filter_])
    up_std_literature = np.delete(up_std_literature, np.where(filter_))
    up_std_literature = np.append(up_std_literature, up_std)
    
mean_literature = mean_literature.reshape((num_marker,)+mean_literature.shape)
up_std_literature = up_std_literature.reshape((num_marker,)+up_std_literature.shape)
down_std_literature = down_std_literature.reshape((num_marker,)+down_std_literature.shape)

In [9]:
# Principal loop, should take a while
dens_markers, std_markers = place_cells(annotation, uniques, children, 
                                        rv, neu_dens, num_neurons,
                                        markers_intensity, alphas, std_fitt, 
                                        names, mean_literature, down_std_literature, up_std_literature,
                                        id_to_region_dictionary_ALLNAME, is_leaf, id_to_region_dictionary)

[====================================================================================================] 100%


In [10]:
# dens_markers is in cell/voxel, convert in cell per mm3
dens_markers /= voxel_volume
for i_marker, name in enumerate(marker_names):
    nrrd.write(join(OUTPUT_FOLDER, "densities_" + name + ".nrrd"), dens_markers[i_marker], header=h)
    with open(join(OUTPUT_FOLDER, "std_" + name + ".json"), 'w') as fp:
        json.dump(std_markers[i_marker], fp, indent=4)